In [2]:
import sys
import numpy as np
import cv2

In [12]:
cap = cv2.VideoCapture('./PETS2000.avi')

# cap.isOpened()
if not cap.isOpened():
    print('Video open failed')
    sys.exit()
    
ret_b, back = cap.read()
back = cv2.cvtColor(back, cv2.COLOR_BGR2GRAY)
back = cv2.GaussianBlur(back, (0, 0), 1.0)
fback = back.astype(np.float32)

if not ret_b:
    print('Back read failed')
    sys.exit()
    
while True:
    
    ret, frame = cap.read()
    
    if not ret:
        print('Frame read failed')
        break
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (0, 0), 1.0)
    
    cv2.accumulateWeighted(gray, fback, 0.01) # float32
    back = fback.astype(np.uint8)
    
    diff = cv2.absdiff(gray, back)
    _, diff = cv2.threshold(diff, 30, 255, cv2.THRESH_BINARY)
    
    ## 
    cnts, labels, stats, centroids = cv2.connectedComponentsWithStats(diff)
    
    for i in range(1, cnts):
        x, y, w, h, s = stats[i]
        
        if s < 200:
            continue
        cv2.rectangle(frame, (x, y, w, h), (0, 0, 255), 2)

    cv2.imshow('frame', frame)
    # cv2.imshow('gray', gray)
    cv2.imshow('diff', diff)
    cv2.imshow('back', back)
    
    
        
    if cv2.waitKey(20) == 27:
        break


cap.release()
cv2.destroyAllWindows()

In [4]:
import sys
import numpy as np
import cv2

cap = cv2.VideoCapture('./Billard_cut.mp4')

if not cap.isOpened():
    print('video open failed')
    sys.exit()
    
    
ret, frame = cap.read()
(x, y, w, h) = cv2.selectROI('roi', frame, showCrosshair = False)
rc = (x, y, w, h)
roi = frame[y:y+h, x:x+w]
roi_hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

channels = [0, 1]
ranges = [0, 180, 0, 256]
hist = cv2.calcHist([roi_hsv], channels, None, [90, 128],  ranges)
term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)

while True:
    
    ret, frame = cap.read()
         
    if not ret:
        print('frame read failed')
        break
    
    frame_hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    backproj = cv2.calcBackProject([frame_hsv], channels, hist, ranges, 1)
    
    ##
    _, rc = cv2.meanShift(backproj, rc, term_crit)
    
    cv2.rectangle(frame, rc, (0, 0, 255), 2)
    cv2.imshow('frame', frame)
    
    if cv2.waitKey(20) == 27:
        break

cap.release()
cv2.destroyAllWindows()